In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import joblib # Used to save the model
import os

# 1. Load Data
if not os.path.exists('conveyor_data.csv'):
    print("Error: 'conveyor_data.csv' not found.")
    exit()

df = pd.read_csv('conveyor_data.csv')
X = df[['Temperature', 'Sway', 'Speed']]
y = df['Label']

# 2. Train Model
print("Training the ARIMA-IoT Intelligence...")
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

# 3. Save the Brain
joblib.dump(model, 'arima_model.joblib')
print("\n" + "="*30)
print("SUCCESS: Model is Ready!")
print("Saved as: arima_model.joblib")
print("="*30)

Training the ARIMA-IoT Intelligence...

SUCCESS: Model is Ready!
Saved as: arima_model.joblib


In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# 1. Create Input Sliders
temp_slider = widgets.FloatSlider(value=25, min=10, max=60, step=0.5, description='Temp (°C):')
sway_slider = widgets.FloatSlider(value=150, min=50, max=250, step=1, description='Sway (cm):')
speed_slider = widgets.IntSlider(value=250, min=0, max=500, step=5, description='Speed (RPM):')

# 2. Create a Visual "Status Light"
status_light = widgets.HTML(value="<div style='width: 50px; height: 50px; background-color: gray; border-radius: 50%;'></div>")
status_text = widgets.Label(value="System Initializing...")

# 3. The Dashboard Logic
def update_dashboard(*args):
    t, s, spd = temp_slider.value, sway_slider.value, speed_slider.value
    
    # AI Prediction
    data = pd.DataFrame([[t, s, spd]], columns=['Temperature', 'Sway', 'Speed'])
    prediction = model.predict(data)[0]
    
    # Update UI based on Anomaly
    if prediction == 0:
        color = "#28a745" # Green
        msg = f"STATUS: HEALTHY | Sway is stable."
    else:
        color = "#dc3545" # Red
        side = "LEFT" if s < 130 else "RIGHT" if s > 170 else "THERMAL"
        msg = f"STATUS: FAULTY | Danger: {side}"

    status_light.value = f"<div style='width: 50px; height: 50px; background-color: {color}; border-radius: 50%; border: 2px solid black;'></div>"
    status_text.value = msg

# 4. Link sliders to the function
temp_slider.observe(update_dashboard, 'value')
sway_slider.observe(update_dashboard, 'value')
speed_slider.observe(update_dashboard, 'value')

# 5. Layout the Dashboard
input_box = widgets.VBox([temp_slider, sway_slider, speed_slider])
output_box = widgets.HBox([status_light, status_text], layout=widgets.Layout(align_items='center', margin='0 0 0 20px'))

print("=== ARIMA-IoT REAL-TIME DASHBOARD ===")
display(widgets.HBox([input_box, output_box]))

# Initial run to set the first state
update_dashboard()

=== ARIMA-IoT REAL-TIME DASHBOARD ===


In [3]:
import pandas as pd
import joblib
import os

# Function to determine specific drift direction and danger zone
def get_diagnosis(t, s, spd, model, last_s):
    # AI Prediction
    current_data = pd.DataFrame([[t, s, spd]], columns=['Temperature', 'Sway', 'Speed'])
    prediction = model.predict(current_data)[0]
    
    # Drift Logic (Comparing to previous input)
    drift = "Stable"
    if last_s is not None:
        if s < last_s: drift = "Right (Towards Sensor)"
        elif s > last_s: drift = "Left (Away from Sensor)"

    # Result Formatting
    status = "HEALTHY" if prediction == 0 else "FAULTY"
    danger_side = "None"
    temp_condition = "Stable"
    if prediction == 1:
        if s < 130: danger_side = "RIGHT (Near Sensor)"
        elif s > 170: danger_side = "LEFT (Off-Track)"
        if t < 20: temp_condition = "Low Temperature"
        elif t > 40: temp_condition = "High Temperature"

    return status, danger_side, drift, temp_condition

# --- MAIN EXECUTION ---
if not os.path.exists('arima_model.joblib'):
    print("Error: Train the model first")
    exit()

# Load the saved brain
trained_model = joblib.load('arima_model.joblib')
last_sway = None

print("\n--- ARIMA-IoT LIVE MONITOR ACTIVE ---")
while True:
    try:
        user_input = input("\nEnter Data [Temp, Sway, Speed] (or 'exit'): ")
        if user_input.lower() == 'exit': break
        
        temp, sway, speed = map(float, user_input.split(','))
        
        status, danger, drift, thermal = get_diagnosis(temp, sway, speed, trained_model, last_sway)
        
        print(f"RESULT: {status}")
        print(f"DANGER ZONE: {danger}")
        print(f"DRIFTING: {drift}")
        print(f"THERMAL STATUS: {thermal}")
        print("-" * 35)
        
        last_sway = sway # Update for next trend check

    except ValueError:
        print("Invalid Format. Please use: 25, 140, 255")


--- ARIMA-IoT LIVE MONITOR ACTIVE ---



Enter Data [Temp, Sway, Speed] (or 'exit'):  exit
